In [9]:
from llmexp.llm.smollm import LLMWrapper
from accelerate import Accelerator
import torch

# checkpoint = "meta-llama/Meta-Llama-3-8B-Instruct"
checkpoint = "meta-llama/Llama-3.2-1B-Instruct"
# checkpoint = "HuggingFaceTB/SmolLM-1.7B-Instruct"
saved_mab_model = "checkpoints/mab_model_100.pth"


accelerator = Accelerator()
device = accelerator.device


llm = LLMWrapper(checkpoint, device=device)
tokenizer = llm.tokenizer

In [10]:
# instruction = "Analyze the sentiment of the following sentence. Be brief."
instruction = "Analyze the sentiment of the following sentence and respond with only one word: 'positive,' 'negative,' or 'neutral,' based on the overall tone and meaning of the sentence. Do not provide any additional explanation."
# user_input = "I am extremely disappointed with the quality; it broke after just one day."
user_input = "I love this movie!"

content = [
            {"role": "system", 
            "content": instruction
            },

            {"role": "sentence", 
            "content": user_input
            }
        ]
template = tokenizer.apply_chat_template(content, tokenize=False, add_generation_prompt=True)
# print(template)

# The generated outputs 
gen_output = llm.generate_from_texts(template)
print(gen_output)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

Analyze the sentiment of the following sentence and respond with only one word: 'positive,' 'negative,' or 'neutral,' based on the overall tone and meaning of the sentence. Do not provide any additional explanation.<|eot_id|><|start_header_id|>sentence<|end_header_id|>

I love this movie!<|eot_id|><|start_header_id|>assistant<|end_header_id|>

positive<|eot_id|>


In [11]:
from llmexp.explainer.mab_model import MABModel
mab_model = MABModel.load_with_base_model(torch.load(saved_mab_model), llm, hidden_size=1024)
mab_model.to(device)
print()

/tmp/1104405.1.gpu/ipykernel_2163222/3922162976.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  mab_model = MABModel.load_with_base_model(torch.load(saved_mab_model), ll

In [12]:
gen_inputs = tokenizer(gen_output, return_tensors="pt").to(device)
input_ids = gen_inputs['input_ids'][:, :-1]
attention_mask = gen_inputs['attention_mask'][:, :-1]
_, mab_values, _ = mab_model.get_dist_value(input_ids, attention_mask)



In [13]:
print(mab_values)
print(mab_values.shape)

tensor([[-0.2145, -0.2145, -0.2145, -0.2028, -0.2089, -0.1987, -0.2045, -0.2113,
         -0.2019, -0.2115, -0.2146, -0.2252, -0.1911, -0.1681, -0.1924, -0.2096,
         -0.2083, -0.2227, -0.2078, -0.2124, -0.1979, -0.2099, -0.2121, -0.2162,
         -0.1573, -0.2019, -0.2094, -0.2091, -0.2048, -0.2144, -0.2027, -0.2063,
         -0.2110, -0.2171, -0.2113, -0.2132, -0.1930, -0.2086, -0.2014, -0.2093,
         -0.2025, -0.2123, -0.2025, -0.1978, -0.2100, -0.2092, -0.2104, -0.2099,
         -0.2052, -0.2093, -0.2043, -0.2118, -0.2111, -0.2104, -0.2141, -0.2137,
         -0.2080, -0.2108, -0.2098, -0.2058, -0.2098, -0.2079, -0.2165, -0.1957,
         -0.1968, -0.2116, -0.2134, -0.2105, -0.2044, -0.2049, -0.1964, -0.2007,
         -0.1989, -0.1991, -0.1970, -0.2129, -0.2133, -0.2223, -0.2092, -0.2119,
         -0.1943, -0.2039, -0.1932, -0.1906, -0.2000]], device='cuda:0',
       grad_fn=<NegBackward0>)
torch.Size([1, 85])


In [14]:
print(input_ids)
print(input_ids.shape)

tensor([[128000, 128000, 128000, 128006,   9125, 128007,    271,  38766,   1303,
          33025,   2696,     25,   6790,    220,   2366,     18,    198,  15724,
           2696,     25,    220,   1627,  10263,    220,   2366,     19,    271,
           2127,  56956,    279,  27065,    315,    279,   2768,  11914,    323,
           6013,    449,   1193,    832,   3492,     25,    364,  31587,   2965,
            364,  43324,   2965,    477,    364,  60668,   2965,   3196,    389,
            279,   8244,  16630,    323,   7438,    315,    279,  11914,     13,
           3234,    539,   3493,    904,   5217,  16540,     13, 128009, 128006,
          52989, 128007,    271,     40,   3021,    420,   5818,      0, 128009,
         128006,  78191, 128007,    271,  31587]], device='cuda:0')
torch.Size([1, 86])


In [15]:
def visualize_tokens_with_values(input_ids, mab_values, tokenizer):
    # Decode tokens one by one to preserve alignment
    tokens = []
    for i in range(input_ids.shape[1]):
        token = tokenizer.decode(input_ids[0, i:i+1])
        tokens.append(token)
    
    # Normalize MAB values to [0,1] for color intensity first
    normalized_values = (mab_values[0] - mab_values[0].min()) / (mab_values[0].max() - mab_values[0].min())
    
    # Pad normalized_values with a zero at the end
    padded_normalized_values = torch.cat([normalized_values, torch.zeros(1, device=mab_values.device)], dim=0)
    # Pad original mab_values with the last actual value
    padded_mab_values = torch.cat([mab_values[0], mab_values[0][-1:]], dim=0)
    
    # Generate HTML with colored text and values
    html_output = "<div style='font-family: monospace; line-height: 2; background-color: white; padding: 10px;'>"
    for token, value, orig_value in zip(tokens, padded_normalized_values, padded_mab_values):
        # Use a gradient from white to green
        intensity = float(value)
        green_color = int(intensity * 200)  # Control the maximum intensity
        html_output += f'<span style="color: black; background-color: rgba(0, {green_color}, 0, 0.3); padding: 0.2em; margin: 0.1em; border-radius: 3px;" title="MAB: {orig_value:.3f}, Norm: {value:.3f}">{token}</span>'
    html_output += "</div>"
    
    # Print the values
    print("Token\tNormalized Value\tOriginal MAB Value")
    print("-" * 50)
    for token, value, orig_value in zip(tokens, padded_normalized_values, padded_mab_values):
        print(f"{token}\t{value:.3f}\t\t{orig_value:.3f}")
    
    from IPython.display import HTML
    return HTML(html_output)

In [16]:
# Usage:
visualization = visualize_tokens_with_values(input_ids, mab_values, tokenizer)
display(visualization)

Token	Normalized Value	Original MAB Value
--------------------------------------------------
<|begin_of_text|>	0.158		-0.214
<|begin_of_text|>	0.158		-0.214
<|begin_of_text|>	0.158		-0.214
<|start_header_id|>	0.331		-0.203
system	0.240		-0.209
<|end_header_id|>	0.390		-0.199


	0.305		-0.205
Cut	0.206		-0.211
ting	0.344		-0.202
 Knowledge	0.203		-0.211
 Date	0.156		-0.215
:	0.000		-0.225
 December	0.503		-0.191
 	0.841		-0.168
202	0.484		-0.192
3	0.231		-0.210

	0.249		-0.208
Today	0.037		-0.223
 Date	0.256		-0.208
:	0.189		-0.212
 	0.402		-0.198
26	0.226		-0.210
 Jul	0.194		-0.212
 	0.133		-0.216
202	1.000		-0.157
4	0.343		-0.202


	0.233		-0.209
An	0.237		-0.209
alyze	0.302		-0.205
 the	0.160		-0.214
 sentiment	0.333		-0.203
 of	0.278		-0.206
 the	0.210		-0.211
 following	0.120		-0.217
 sentence	0.205		-0.211
 and	0.178		-0.213
 respond	0.475		-0.193
 with	0.245		-0.209
 only	0.352		-0.201
 one	0.235		-0.209
 word	0.334		-0.203
:	0.191		-0.212
 '	0.334		-0.203
positive	0.404		-0.198
